In [ ]:
import sys
import os
import datetime

base_folder = r'D:\measuring'

if not base_folder in sys.path:
    sys.path.append(base_folder)
execfile(os.path.join(base_folder,r'analysis\scripts\setup_analysis.py'))
execfile(os.path.join(base_folder,r'analysis\scripts\bell\setup_bell_analysis.py'))

%matplotlib inline

### Process new Bell data

In [ ]:
#bs_folder = r'D:\measuring\data\2015-03-03-lhfbt\BS'
#measurement_pattern = 'first_ever'
#bs_folder = r'D:\measuring\data\2015-03-18-lhfbt2\BS'
#measurement_pattern = 'second_ever'
#bs_folder = r'D:\measuring\data\2015-04-13-lhfbt3\BS'
#measurement_pattern = 'third_ever'
#bs_folder = r'D:\measuring\data\2015-04-21-lhfbt4\BS'
#measurement_pattern = 'TheFourth'
#bs_folder = r'D:\measuring\data\2015-05-15-ZZ\BS'
#measurement_pattern = 'BackToZZ'
#bs_folder = r'D:\measuring\data\2015-06-10_XX\BS'
#measurement_pattern = 'finallyXX'
#bs_folder = r'D:\measuring\data\2015-06-28-lhfbt5\BS'
#measurement_pattern = 'TheFinal'
#bs_folder = r'D:\measuring\data\2015-11-16_XX\BS'
#measurement_pattern = 'moreXX'
#bs_folder = r'D:\measuring\data\2015-12-01-lhfbt6\BS'
#measurement_pattern = 'TheSecondFinal'
bs_folder = r'D:\measuring\data\2015-12-08-lhfbt6-new_detector\BS'
measurement_pattern = 'TheSecondFinal'
#lt3_folder = r'Y:\data'
#lt4_folder = r'X:\data'
lt3_folder = os.path.join(os.path.split(bs_folder)[0],'LT3')
lt4_folder = os.path.join(os.path.split(bs_folder)[0],'LT4')
try:
    settings = imp.load_source('bell_analysis_settings',os.path.join(os.path.split(bs_folder)[0],'bell_analysis_settings.py'))
except IOError:
    print 'Data specific analysis settings not found, using standard settings!'
    pass

update_previous_analysis = False

output_folder = os.path.split(bs_folder)[0]
analysis_fp = os.path.join(output_folder,tb.get_timestamp_from_now()+'_'+'total_events.hdf5')
previous_analysis_fp = bell_data.get_latest_analysis_fp(output_folder, pattern ='total_events.hdf5') if update_previous_analysis else None

In [ ]:
fps_bs = tb.get_all_msmt_filepaths(bs_folder, pattern=measurement_pattern)
bell_data.process_bell_data(fps_bs, lt3_folder, lt4_folder, 
                            settings.bs_process_params, settings.lt_process_params,
                            analysis_fp, update_previous_analysis_fp = previous_analysis_fp, 
                            ignore_unequal_markers   = 'fix_last', 
                            process_lt3 = True, process_lt4 = True, 
                            VERBOSE=False)
print ''

##### ------------------- 

In [ ]:
#here we get a list of files that actually contributed to the data ie had at least one entanglement event
bs_fps, lt3_fps, lt4_fps = bell_data.get_unique_bell_fps_from_analysis_file(analysis_fp)

In [ ]:
#here we get the tail & pulse counts per run
bell_data.process_bs_hist_stats(bs_fps, settings.analysis_params ,analysis_fp, update_previous_analysis_fp = previous_analysis_fp)
print ''

In [ ]:
bell_data.process_tpqi_data(bs_fps, settings.bs_process_params, analysis_fp,  
                            update_previous_analysis_fp = previous_analysis_fp)
print ''

In [ ]:
bell_data.process_lt_stats(lt3_fps, settings.lt_process_params, lt3=True, 
                           analysis_fp=analysis_fp, update_previous_analysis_fp = previous_analysis_fp)
bell_data.process_lt_stats(lt4_fps, settings.lt_process_params, lt3=False,
                           analysis_fp=analysis_fp, update_previous_analysis_fp = previous_analysis_fp)
print ''

In [ ]:
bell_data.process_afterpulsing_data(bs_fps, settings.after_pulse_process_params, analysis_fp,  update_previous_analysis_fp = previous_analysis_fp)
print ''

### Plot and analyse processed Bell data 

In [ ]:
#output_folder = r'D:\measuring\data\2015-03-18-lhfbt2'
#output_folder = r'D:\measuring\data\2015-04-13-lhfbt3'
#output_folder = r'D:\measuring\data\2015-04-21-lhfbt4'
#output_folder = r'D:\measuring\data\2015-05-15-ZZ'
#output_folder = r'D:\measuring\data\2015-06-10_XX'
#output_folder = r'D:\measuring\data\2015-06-28-lhfbt5'
#output_folder  = r'D:\measuring\data\2015-12-01-lhfbt6'
output_folder = r'D:\measuring\data\2015-12-08-lhfbt6-new_detector'

analysis_fp = bell_data.get_latest_analysis_fp(output_folder, pattern ='total_events')
#here we get a list of files that actually contributed to the data ie had at least one entanglement event
bs_fps, lt3_fps, lt4_fps = bell_data.get_unique_bell_fps_from_analysis_file(analysis_fp)
print analysis_fp

In [ ]:
#plot tail & pulse counts vs run:
f = h5py.File(analysis_fp,'r')
bs_hist_stats = f['analysis']['bs_hist_stats'].value
f.close()

x = np.arange(len(bs_hist_stats[:,be._cl_bs_tail]))
y_tail  = bs_hist_stats[:,be._cl_bs_tail].astype(np.float)/bs_hist_stats[:,be._cl_bs_last_sync]*1e4
u_y_tail = np.sqrt(bs_hist_stats[:,be._cl_bs_tail].astype(np.float))/bs_hist_stats[:,be._cl_bs_last_sync]*1e4
y_pulse = bs_hist_stats[:,be._cl_bs_pulse].astype(np.float)/bs_hist_stats[:,be._cl_bs_last_sync]*1e4
u_y_pulse = np.sqrt(bs_hist_stats[:,be._cl_bs_pulse].astype(np.float))/bs_hist_stats[:,be._cl_bs_last_sync]*1e4

ax=plt.subplot(111)
ax.errorbar(x,y_tail,yerr=u_y_tail, fmt='x', label='Tail')
ax.errorbar(x,y_pulse,yerr=u_y_pulse, fmt='x', label = 'Pulse')
ax.set_ylabel('Tailcounts * 1e4')
ax.set_ylim(0,6)
ax.set_xlabel('Run #')
ax.set_title('Tail & pulse counts \n'+btools.plot_title(analysis_fp))
#ax.legend()
btools.save_figure('Tail', ax, output_folder, analysis_fp)

In [ ]:
np.sum(bs_hist_stats[:,be._cl_bs_last_time]*1e-12)/60./60.

In [ ]:
#plot summed histogram:
f = h5py.File(analysis_fp,'r')
bs_total_hist = f['analysis']['bs_total_hist'].value
bs_hist_stats = f['analysis']['bs_hist_stats'].value
f.close()
hist_binsize = settings.bs_process_params['hist_binsize_ps']*1e-12
xmin = 54150
xmax = 54800
x = np.arange(xmin,xmax)*hist_binsize *1e6 #us
y_0 = bs_total_hist[xmin:xmax,0].astype(np.float)
y_1 = bs_total_hist[xmin:xmax,1].astype(np.float)
ax=plt.subplot(111)
ax.semilogy(x,y_0,label='channel 0')
ax.semilogy(x+.0006,y_1,label='channel 1')
ax.set_title('Total histogram \n'+btools.plot_title(analysis_fp))
ax.set_ylabel('Counts')
ax.set_xlabel('time [us]')
ax.grid(True)
#ax.legend()
btools.save_figure('Total_hist', ax, output_folder, analysis_fp)

#total measurement time
#figure()
#ax=plt.subplot(111)
#ax.hist(bs_hist_stats[:,be._cl_bs_last_time]/1e12/60.)
print float(np.sum(bs_hist_stats[:,be._cl_bs_last_time]))/1e12/60./60.

In [ ]:
#plot a histogram of a specific run(s)
#fp = bs_fps[-1]
#fp = bs_fps[-1]
fp =tb.get_msmt_fp(tb.latest_data(contains='day3_run11', folder=r'Z:\data'))
pqf = h5py.File(fp,'r')
pq_plots.plot_marker_filter_comparison(pqf,
                          mrkr_chan = 1,
                          start = 5420e3,#settings.bs_process_params['st_start_ch0'],  
                          length= 10e3,#settings.bs_process_params['st_len']+settings.bs_process_params['pulse_sep'], 
                          hist_binsize = .25e3, save = False, log=False) #start =7480
pqf.close() 

In [ ]:
#plot the TPQI

In [ ]:
f = h5py.File(analysis_fp,'r')
coincidences = f['analysis']['tpqi'].value
f.close()

st_start_ch0 = settings.analysis_params['st_start_ch0']
st_len       = settings.analysis_params['st_len']
st_start_ch1 = settings.analysis_params['st_start_ch1']
p_sep        = settings.analysis_params['pulse_sep']
st0=coincidences[:,be._cl_tpqi_st0]
st1=coincidences[:,be._cl_tpqi_st1]
st_fltr_0 = (((st_start_ch0<=st0)  & (st0<(st_start_ch0+st_len))) | ((st_start_ch0+p_sep<=st0) & (st0<(st_start_ch0+p_sep+st_len))) )  
st_fltr_1 = (((st_start_ch1<=st1)  & (st1<(st_start_ch1+st_len))) | ((st_start_ch1+p_sep<=st1) & (st1<(st_start_ch1+p_sep+st_len))) )
coincidences_fltr=coincidences[st_fltr_0 & st_fltr_1 ,be._cl_tpqi_dt]/1000.
dt=100
center_peak = np.sum((coincidences_fltr>-dt) & (coincidences_fltr<dt))
#print ii
left_peak  = np.sum((coincidences_fltr>(-dt-p_sep/1000)) & (coincidences_fltr<(dt-p_sep/1000)))
right_peak = np.sum((coincidences_fltr>(-dt+p_sep/1000)) & (coincidences_fltr<(dt+p_sep/1000)))
Vis = 1- float(center_peak)/(left_peak+right_peak)
ax=plt.subplot(111)
bins=np.linspace(-450,450,80)
hist_TPQI, bins_TPQI, patches_TPQI = ax.hist(coincidences_fltr,bins=bins, color='b', cumulative=False, histtype = 'step')
ax.set_title('TPQI histogram, V = {:.1f} % \n {}'.format(Vis*100, btools.plot_title(analysis_fp)))
ax.set_ylabel('Counts',fontsize = 15)
ax.set_ylim(0,1.1*max(hist_TPQI))
ax.tick_params(axis = 'y', labelsize = 13)
ax.set_xlabel('Time [ns]',fontsize = 15)
ax.tick_params(axis = 'x', labelsize = 13)
ax.text(-p_sep/1000,max(hist_TPQI),left_peak, horizontalalignment='center', verticalalignment ='bottom')
ax.text(+p_sep/1000,max(hist_TPQI),right_peak, horizontalalignment='center', verticalalignment ='bottom')
ax.text(0,max(hist_TPQI),center_peak, horizontalalignment='center', verticalalignment ='bottom')
btools.save_figure('TPQI', ax, output_folder, analysis_fp)

In [ ]:
#Plot the PSB SP Corrs vs run #
for setupname in ['lt3','lt4']:
    f = h5py.File(analysis_fp,'r')
    lt_stats = f['analysis'][setupname+'_stats'].value
    f.close()
    figure()
    ax=plt.subplot(111)
    ax.set_title('PSB SP correlations '+ setupname +'\n'+ btools.plot_title(analysis_fp))
    ax.plot(lt_stats[:,be._cl_lt_p_w1_ro0_rnd0],label = 'rnd0, w1')
    ax.plot(lt_stats[:,be._cl_lt_p_w1_ro0_rnd1],label = 'rnd1, w1')
    ax.plot(lt_stats[:,be._cl_lt_p_w2_ro0_rnd0],label = 'rnd0, w2')
    ax.plot(lt_stats[:,be._cl_lt_p_w2_ro0_rnd1],label = 'rnd1, w2')
    ax.plot(lt_stats[:,be._cl_lt_noof_w1]/(lt_stats[:,be._cl_lt_noof_w1]+lt_stats[:,be._cl_lt_noof_w2]), label='w1/(w1+w2)')
    ax.set_xlabel('Run #')
    ax.set_ylabel('Prob ms = 0, uncorrected')
    ax.legend()
    btools.save_figure('Window_correlations_'+setupname, ax, output_folder, analysis_fp)

    #Plot Tail & prepulse vs run
    figure()
    ax=plt.subplot(111)
    ax2=ax.twinx()
    ax.set_title('PSB Tail '+ setupname +'\n'+btools.plot_title(analysis_fp))
    ax.plot(lt_stats[:,be._cl_lt_tail]*1e4,label = 'Tail')
    ax2.plot(lt_stats[:,be._cl_lt_prepulse]*1e4,'g',label = 'Prepulse')
    ax.set_xlabel('Run #')
    ax.set_ylabel('Tail [counts x 1e4]')
    ax2.set_ylabel('Prepulse [counts x 1e4]')
    ax.legend()
    btools.save_figure('PSB_tail_'+setupname, ax, output_folder, analysis_fp)

In [ ]:
#Plot a histogram of the PSB data #TBD

In [ ]:
#analyse Bell data correlations
#reload(settings)
reload(btools)
f = h5py.File(analysis_fp,'r')
db_fps=f['analysis']['total_ent_events_fps'].value
db = f['analysis']['total_ent_events'].value
d3 = f['analysis']['total_lt3_ssro'].value
d4 = f['analysis']['total_lt4_ssro'].value
f.close()
#bad_fps=bs_fps[ys>2500]
#fltr= np.array([(fp not in bad_fps) for fp in db_fps]); 
#fltr = np.array([('day9_run20' not in fp) for fp in db_fps])
#db=db[fltr]; d3=d3[fltr]; d4=d4[fltr]
#settings = imp.load_source('bell_analysis_settings',os.path.join(os.path.split(bs_folder)[0],'bell_analysis_settings.py'))
ap=settings.analysis_params
#ap['invalid_marker_max_sn_diff']=10
#ap['ro_length']=3700
#print ap['st_start_ch0']
rn=-1
corr_mats, _tmp = btools.get_corr_mats(db[:rn],d3[:rn],d4[:rn],db_fps[:rn], ap, bad_time_ranges.bad_time_ranges, VERBOSE=True)

In [ ]:
corr_mats_sum={}
for psi in corr_mats:
    corr_mats_sum[psi] = corr_mats[psi]+corr_mats_day_1_to_5[psi]

In [ ]:
#corr_mats_day_1_to_5 = corr_mats
np.sqrt(0.034)

In [ ]:
print btools.plot_title(analysis_fp)
CHSH, dCHSH, Es, dEs = btools.print_correlators(corr_mats,psis=['psi_min','psi_plus'])

In [ ]:
reload(btools)
CHSH, dCHSH, Es, dEs = btools.print_correlators_combined(corr_mats_sum)

In [ ]:
print btools.plot_title(analysis_fp)
btools.calculate_p_lhv(corr_mats)
print ''

In [ ]:
print btools.plot_title(analysis_fp)
btools.print_XX_fidelity(corr_mats, settings.analysis_params)

In [ ]:
print btools.plot_title(analysis_fp)
btools.print_ZZ_fidelity(corr_mats, settings.analysis_params)

#### -----------------------------------------------

In [ ]:
X, S, dS, P, Ns, Ks = btools.sweep_analysis_param(analysis_fp,settings,[],['st_len_w2_00','st_len_w2_11'],-2000,20000,50,True)

In [ ]:
ax1 = plt.subplot(311)
ax1.plot(X,(Ks.astype(np.float)/Ns-0.5)*8)
ax1.set_xticks([])
ax2=plt.subplot(312)
ax2.plot(X,Ns,'r')
ax2.set_xticks([])
ax3=plt.subplot(313)
ax3.plot(X,P)

In [ ]:
#plot data correlations
pts=5
X=np.linspace(-2000,2000,pts)+5426000
Y=np.zeros(pts)
for i,x in enumerate(X):
    ap=settings.analysis_params.copy()
    ap['st_start_ch0']=x
    ap['st_start_ch1']=x-900
    corr_mats, _tmp = btools.get_corr_mats(db,d3,d4,db_fps, ap, bad_time_ranges.bad_time_ranges, VERBOSE=False)
    K,N,Kxx,Nxx,Kzz,Nzz,p_val = btools.calculate_p_lhv(corr_mats, VERBOSE=False)
    CHSH, dCHSH, Es, dEs = btools.print_correlators(corr_mats, VERBOSE=False, psis=['psi_min'])
    Y[i] = CHSH
plt.plot(X,Y)

In [ ]:
fltr = (d3[:,be._cl_sn_ma] > 0) & (d4[:,be._cl_sn_ma] > 0) & db[:,be._cl_type] == 1
td = db[fltr,be._cl_tt_w1].astype(np.float)
t4 = d4[fltr,be._cl_tt_rnd].astype(np.float)
t3 = d3[fltr,be._cl_tt_rnd].astype(np.float)
sd = db[fltr,be._cl_sn].astype(np.float)
s4 = d4[fltr,be._cl_sn_ma].astype(np.float)
s3 = d3[fltr,be._cl_sn_ma].astype(np.float)
#tdiff3 = td/1e12 - t3/1e9
#tdiff4 = td/1e12 - t4/1e9
tdiff3 = sd/251. - s3/250.
tdiff4 = sd/251. - s4/251.
#figure()
hist(tdiff3, bins = np.linspace(-1,1,50))
hist(tdiff4)#, bins = np.linspace(-1,20,50))
print np.min(tdiff3), max(tdiff3)
print min(tdiff4), max(tdiff4)
print np.argmax(tdiff3), np.argmin(tdiff3)
print np.argmax(tdiff4), np.argmin(tdiff4)
#print db_fps[fltr][np.abs(tdiff3)>1]
print tdiff3[np.abs(tdiff3)>1]

In [ ]:
diff3 = d3[fltr,be._cl_tt_ma].astype(np.float) - d3[fltr,be._cl_tt_rnd].astype(np.float)
diff4 = d4[fltr,be._cl_tt_ma].astype(np.float) - d4[fltr,be._cl_tt_rnd].astype(np.float)
#hist(diff3)
hist(diff4)

In [ ]:
print st_start_ch0

In [ ]:
#bad_fps=bs_fps[ys>2500]
#fltr= np.array([(fp not in bad_fps) for fp in db_fps]); 
#fltr = np.array([('day10' in fp) for fp in db_fps])
#db=db[fltr]; d3=d3[fltr]; d4=d4[fltr]
days=19
fltr=False
Ks = np.zeros(days)
Ns=np.zeros(days)
p_lhvs = np.zeros(days)
X=np.arange(days)
for i in X:
    fltr = fltr | np.array([('day'+str(i+1)+'_' in fp) for fp in db_fps])
    dbf=db[fltr]; d3f=d3[fltr]; d4f=d4[fltr]
    corr_mats, _tmp = btools.get_corr_mats(dbf,d3f,d4f,db_fps, settings.analysis_params, bad_time_ranges.bad_time_ranges, VERBOSE=False)
    K,N,Kxx,Nxx,Kzz,Nzz,p_lhv = btools.calculate_p_lhv(corr_mats, VERBOSE=False)
    Ks[i] = K
    Ns[i] = N
    p_lhvs[i] = p_lhv

In [ ]:
ax=plt.subplot(311)
ax.plot(X+1,(Ks.astype(np.float)/Ns-0.5)*8, 'r', label='S')
ax.set_ylabel('CHSH value')
ax.set_ylim(1,3)
ax.grid(True)
ax.set_xticklabels([])
ax2=plt.subplot(312)
ax2.plot(X+1,Ns, label='N')
ax2.set_ylabel('N')
ax2.grid(True)
ax2.set_xticklabels([])
ax3=plt.subplot(313)
ax3.plot(X+1,p_lhvs, label='p-value')
ax3.set_ylabel('p-value')
ax3.grid(True)
ax3.set_xlabel('Measurement day #')

#### -------------------------

In [ ]:
#plot a histogram of a specific run(s)
ax1=plt.subplot(111)
#ax1=plt.subplot(311)
#ax2=plt.subplot(312)
#ax3=plt.subplot(313)
ax1.set_title('Histogram for single run')
ax1.set_ylabel('Counts')
ax1.set_xlabel('time [us]')
hist_binsize = settings.bs_process_params['hist_binsize_ps']*1e-12
xmin = 54350
xmax = 54500
ch1_off = 11

fps=bs_fps[:]#[[1,-1]]
#print fps
ys = np.zeros(len(fps))
for i,fp in enumerate(fps):
    bs_hist, _t = bell_data.process_bs_hist_stats([fp], settings.bs_process_params)
    x = np.arange(xmin,xmax)*hist_binsize *1e6 #us *1e6 #us
    y_0 = bs_hist[xmin:xmax,0].astype(np.float)
    y_1 = bs_hist[xmin+ch1_off:xmax+ch1_off,1].astype(np.float)  
    #ax1.semilogy(x,y_0, label=tb.get_plot_title_from_folder(os.path.split(fp)[0])[-9:])
    #ax2.semilogy(x,y_1)
    #ax3.plot(x,y_0-y_1)
    print np.where(bs_fps == fp)[0], np.sum(np.abs(y_0-y_1))
    ys[i] = np.sum(np.abs(y_0-y_1))
ax.plot(ys)
#ax.legend()

In [ ]:
reload(btools)

analysis_fp = bell_data.get_latest_analysis_fp(output_folder, pattern ='total_events')
settings = imp.load_source('bell_analysis_settings',os.path.join(os.path.split(bs_folder)[0],'bell_analysis_settings.py'))

f = h5py.File(analysis_fp,'r')
db_fps=f['analysis']['total_ent_events_fps'].value
db = f['analysis']['total_ent_events'].value
d3 = f['analysis']['total_lt3_ssro'].value
d4 = f['analysis']['total_lt4_ssro'].value
f.close()
#rn=-110
#db=db[:rn];d3=d3[:rn];d4=d4[:rn];db_fps=db_fps[:rn]
#bad_fps=bs_fps[ys>2500]
#fltr= np.array([(fp not in bad_fps) for fp in db_fps]); 
#fltr = np.array([('day1_run7' not in fp) for fp in db_fps])
#db=db[fltr]; d3=d3[fltr]; d4=d4[fltr]

#psi_name = 'psi_min'
#corr_mats, fltr = btools.get_corr_mats(db,d3,d4,db_fps, settings.analysis_params, bad_time_ranges.bad_time_ranges, ret_fltr_psi_name=psi_name, VERBOSE=False)
#print np.sum(fltr)
valid_event_fltr = db[:,be._cl_type] == 1
rnd_fltr = (d3[:,be._cl_noof_rnd_0] + d3[:,be._cl_noof_rnd_1] == 1 ) \
                 & (d4[:,be._cl_noof_rnd_0] + d4[:,be._cl_noof_rnd_1] == 1 ) 
fltr=valid_event_fltr & rnd_fltr


st_w1=db[fltr][:,be._cl_st_w1]
st_w2=db[fltr][:,be._cl_st_w2]
ch_w1=db[fltr][:,be._cl_ch_w1]
ch_w2=db[fltr][:,be._cl_ch_w2]
rnd3= (d3[fltr][:,be._cl_noof_rnd_1])
rnd4= np.logical_not(d4[fltr][:,be._cl_noof_rnd_1])
rnd3_time = d3[fltr][:,be._cl_st_rnd]
rnd4_time = d4[fltr][:,be._cl_st_rnd]
ro3_time = d3[fltr][:,be._cl_first_ph_st]
ro4_time = d4[fltr][:,be._cl_first_ph_st]
last_invalid3 = d3[fltr][:,be._cl_last_inv_mrkr]
last_invalid4 = d4[fltr][:,be._cl_last_inv_mrkr]
tail3_time = d3[fltr][:,be._cl_first_tail_st]
tail4_time = d4[fltr][:,be._cl_first_tail_st]
days=[]
runs=[]
event_times=[]
for i in range(len(db[fltr])):
         
        event_times.append(tb.get_datetime_from_folder(os.path.split(db_fps[fltr][i])[0]) \
                    + datetime.timedelta(seconds = db[fltr][i,be._cl_tt_w1]*1e-12))
        s=db_fps[fltr][i]
        try:
            i_day=s.find('_day',-30,-1)
            i_run=s.find('_run',-30,-1)
            i_hdf=s.find('.hdf',-30,-1)
            days.append(int(s[i_day+4:i_run]))
            runs.append(int(s[i_run+5:i_hdf]))
        except Exception as e:
            print s
            print e
            raise Exception('')

arr = np.vstack((event_times,days,runs,st_w1,ch_w1,st_w2,ch_w2,rnd3,rnd4,rnd3_time,rnd4_time,ro3_time,ro4_time,tail3_time,tail4_time,last_invalid3,last_invalid4))
np.savetxt(btools.save_fp(output_folder,analysis_fp)+'_'+'.txt',arr.T,delimiter=',', fmt=['%s','%u','%u','%u','%u','%u','%u','%u','%u','%u','%u','%u','%u','%u','%u','%u','%u'])

In [ ]:
s=db_fps[2000]
i_day=s.find('_day',-30,-1)
i_run=s.find('_run',-30,-1)
i_hdf=s.find('.hdf',-30,-1)

print s[i_day+4:i_run]
print s[i_run+4:i_hdf]

In [ ]:
i_hdf

In [ ]:
np.loadtxt(btools.save_fp(output_folder,analysis_fp)+'_'+'.txt',delimiter=',', usecols=arange(1,15), dtype=np.uint64)

In [ ]:
np.loadtxt(btools.save_fp(output_folder,analysis_fp)+'_'+'.txt',delimiter=',', usecols=[0], dtype=np.datetime64)